<a href="https://colab.research.google.com/github/LUMII-AILab/NLP_Course/blob/main/notebooks/MSP/Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lielo valodas modeļu novērtēšana

In [ ]:
# Uzninstalējam OpenAi plaformu LVM darbināšanai
!pip install openai

In [ ]:
# Izmantosim 3 modeļus: bāzes modeļus TildeOpen un Gemma-2-27b un pielāgoto modeli gemma-2-27b-it
models = {
    "google/gemma-2-27b-it": "http://gauja.ailab.lv:30051/v1",
    "google/gemma-2-27b": "http://gauja.ailab.lv:30052/v1",
    "TildeAI/TildeOpen-30b": "http://gauja.ailab.lv:30053/v1"
}

##  Vienkāršota LVM darbināšana

In [ ]:
# Vienkāršu uzvedņu piemēri bāzes (my_prompt) un pielāgotajam (my_prompt_instruct) modelim
my_prompt_instruct="Slavens Latvijas komponists ir Raimonds \n A: Pauls \n B: Zirdziņš \n C: Bērziņš \n D: Lakstīgala \n Answer the above question by replying with 'a', 'b', 'c' or 'd', and nothing else. "
my_prompt="Slavens Latvijas populārās mūzikas komponists ir Raimonds"
prefix= "The following are multiple choice questions (with answers)."

prefix_en= "The following are multiple choice questions (with answers) in Latvian."
prefix_lv="Tālāk seko jautājumi ar vairākām atbilžu izvēlēm (ar atbildēm)."

instruction_lv="Atbildiet uz iepriekšējo jautājumu, atbildot ar 'a' vai 'b', un nekas cits."
instruction_en="Answer to the question by replying with '1' or '2' and nothing else."
myprompt="Īrnieki tika izlikti no dzīvokļa.\nIzvēles:\na. Viņi savu īri nemaksāja.\nb. Viņi sapratās ar savu saimnieku."
# instruction_prompt="Question: {text} \n Options: \na. {option_a} \n b. {option_b} \n c. {option_c} \n d. {option_d} \n Answer the above question by replying with 'a', 'b', 'c' or 'd', and nothing else."

In [ ]:
# Bāzes modeļu darbināšana

from openai import OpenAI
# model = "TildeAI/TildeOpen-30b"
model = "google/gemma-2-27b"
client = OpenAI(
    base_url=models[model],
    api_key="any_key",
)
response=client.completions.create(
 #   prompt="Latvijā slavens komponists ir Raimonds",
    prompt=my_prompt,
    max_tokens=10,
    temperature=1,
    model=model
)
print(response.choices[0].text)

In [ ]:
# Pielāgota modeļa darbināšana
# Completion komanda domāta teksta turpināšanai
from openai import OpenAI
model = "google/gemma-2-27b-it"
client = OpenAI(
    base_url=models[model],
    api_key="any_key"
)
response=client.completions.create(
    prompt=my_prompt,
    max_tokens=10,
    temperature=1,
    model=model
)
print(response.choices[0].text)

In [ ]:
# Pielāgota modeļa darbināšana
# chat komanda ļauj labāk iegūt atbildi uz jautājumu
from openai import OpenAI
model = "google/gemma-2-27b-it"
client = OpenAI(
    base_url=models[model],
    api_key="any_key",
)
# Response=client.completions.create
response=client.chat.completions.create(
    model=model,
    messages=[
        {
            "role": "user",
            "content": prefix + my_prompt_instruct,
        },
    ],
)
print(response.choices[0].message.content)

## Novērtēšana ar datu kopām


### COPA

In [ ]:
import json
import requests

url = "https://raw.githubusercontent.com/LUMII-AILab/NLP_Course/main/notebooks/resources/mini-copa.jsonl"

response = requests.get(url)
response.raise_for_status()  # check for HTTP errors
lines = response.text.strip().split("\n")
test_data = [json.loads(line) for line in lines]


from openai import OpenAI
model = "google/gemma-2-27b-it"
client = OpenAI(
    base_url=models[model],
    api_key="any_key",
)
correct=0
total = len(test_data)
for example in test_data:
    my_prompt = "Jautājums:"+example["premise"]+"\nIzvēles: 1. "+example["choice1"]+"\n2. "+example["choice2"]
    correct_answer = example["label"]+1
    print(correct_answer)
    print(my_prompt)
    response=client.chat.completions.create(
      model=model,
      messages=[
        {
            "role": "user",
            "content": prefix_en + my_prompt+ instruction_en,
        },
      ],
      temperature=1,
      max_tokens=10,
    )
    answer=response.choices[0].message.content.strip()
    print(answer)
    if answer==str(correct_answer):
       correct += 1

print(correct)
print(total)
accuracy = correct / total
print("Accuracy:", accuracy)

### LV-Exams datu kopa

In [ ]:
import re
import random
import requests
from openai import OpenAI

model = "google/gemma-2-27b-it"
client = OpenAI(
    base_url=models[model],
    api_key="any_key",
)

questions = requests.get("https://raw.githubusercontent.com/LUMII-AILab/NLP_Course/refs/heads/main/notebooks/resources/lv_exams.json").json()
random.seed(42)
random.shuffle(questions)

def format_question(question):
  text = question["question"] + "\n"
  for k, v in question["options"].items():
    text += f"{k}: {v}\n"

  return text

correct = 0
total = 0
for q in questions[:10]:
  prompt = format_question(q)
  prompt += "Atbildi formātā 'Atbilde ir: X', kur X ir pareizās atbildes burts.\n"


  response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": prompt
                    }
                ]
            }
        ],
        temperature=1,
        max_tokens=10,
    )
  response_text = response.choices[0].message.content.strip()
  m = re.search(r"Atbilde ir[:*\s]*([A-Z])", response_text, re.I | re.U)
  model_answer = m.group(1) if m is not None else "None"
  correct += model_answer == q['answer']
  total += 1
  print(prompt)
  print("-------------")
  print(f"Expected {q['answer']}, full response: {response_text}")
  print("\n========\n")

print("Accuracy:", correct/total)




### BeleBele datu kopa

In [ ]:
import json
import re
import random
import datasets
from openai import OpenAI

model = "google/gemma-2-27b-it"
client = OpenAI(
    base_url=models[model],
    api_key="any_key",
)

def format_question(question):
  text = question["flores_passage"] + "\n"
  text += question["question"] + "\n"
  letters = ['A', 'B', 'C', 'D']
  for i in range(4):
    text += f"{letters[i]}: {question['mc_answer'+str(i+1)]}\n"
    if str(i+1) == question['correct_answer_num']:
      question['answer'] = letters[i]

  return text

dataset = datasets.load_dataset('facebook/belebele', 'lvs_Latn')
correct = 0
total = 0
for q in dataset['test'].select(range(10)):
  prompt = format_question(q)
  prompt += "Atbildi formātā 'Atbilde ir: X', kur X ir pareizās atbildes burts.\n"


  response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": prompt
                    }
                ]
            }
        ],
        temperature=1,
        max_tokens=10,
    )
  response_text = response.choices[0].message.content
  m = re.search(r"Atbilde ir[:*\s]*([A-Z])", response_text, re.I | re.U)
  model_answer = m.group(1) if m is not None else "None"
  correct += model_answer == q['answer']
  total += 1
  print(prompt)
  print("-------------")
  print(f"Expected {q['answer']}, model guessed {model_answer}, full response: {response_text}")
  print("\n========\n")



print("Accuracy:", correct/total)




### 5 piemēru uzvedne

In [ ]:
import re
import random
import requests
from openai import OpenAI

model = "google/gemma-2-27b"
client = OpenAI(
    base_url=models[model],
    api_key="any_key",
)

questions = requests.get("https://raw.githubusercontent.com/LUMII-AILab/NLP_Course/refs/heads/main/notebooks/resources/lv_exams.json").json()
random.seed(42)
random.shuffle(questions)

def format_question(question, with_answer=False):
    text = question["question"] + "\n"
    for k, v in question["options"].items():
      text += f"{k}: {v}\n"

    if with_answer:
       text += f"Atbilde ir: {question['answer']}\n"

    return text

# Number of few-shot examples to use
N_SHOTS = 5

# Generate few-shot prompt from first N items
few_shot_prompt = ""
for q in questions[:N_SHOTS]:
    few_shot_prompt += format_question(q, True)
    few_shot_prompt += "\n"

correct = 0
total = 0

# Test on items after the few-shot examples
for q in questions[N_SHOTS:N_SHOTS+10]:
    prompt = few_shot_prompt + format_question(q)

    response = client.completions.create(
        model=model,
        prompt=prompt,
        temperature=0.9,
        max_tokens=10,
    )

    response_text = response.choices[0].text.strip()
    m = re.search(r"Atbilde ir[:*\s]*([A-Z])", response_text, re.I | re.U)
    model_answer = m.group(1) if m is not None else "None"
    correct += model_answer == q['answer']
    total += 1
    print(prompt)
    print("-------------")
    print(f"Expected {q['answer']}, full response: {response_text}")
    print("\n========\n")

print("Accuracy:", correct/total)

### Logvarbūtību vērtēšana, bez ģenerēšanas



In [14]:
import string, requests, random
from openai import OpenAI

def predict_best_option(base_url, model, question, options, question_tag="Jautājums:", answer_tag="Pareizā atbilde:"):
    client = OpenAI(base_url=base_url, api_key="any_key")
    labels = list(string.ascii_uppercase)[:len(options)]
    prompt = question_tag + question + "\n" + "\n".join(f"{k}) {o}" for k, o in zip(labels, options)) + f"\n{answer_tag}"

    def logp(label):
        r = client.completions.create(
            model=model, prompt=prompt + f" {label}",
            max_tokens=0, echo=True, logprobs=5,
            temperature=0, top_p=1
        )
        return r.choices[0].logprobs.token_logprobs[-1]

    scores = {o: logp(k) for k, o in zip(labels, options)}
    return max(scores, key=scores.get)

questions = requests.get("https://raw.githubusercontent.com/LUMII-AILab/NLP_Course/refs/heads/main/notebooks/resources/lv_exams.json").json()
random.seed(42)
random.shuffle(questions)

# model = "google/gemma-2-27b-it"
# model = "google/gemma-2-27b"
model = "TildeAI/TildeOpen-30b"
correct = 0
sample_questions = questions[:10]
for q in sample_questions:
    pred = predict_best_option(models[model], model, q["question"], list(q["options"].values()))
    pred_label = next(k for k, v in q["options"].items() if v == pred)
    gold_label = q["answer"]
    print(q['question'])
    print(*[k + ': ' + v for k, v in q['options'].items()], sep="\n")
    print(f"Pred: {pred_label}, Gold: {gold_label}\n")
    correct += (pred_label == gold_label)

print("Accuracy:", correct / len(sample_questions))

Kuras iezīmes raksturīgas cunami?
A: Tiem raksturīgi īsi viļņi un liela amplitūda. Parasti pārvietojas ar ātrumu 30 km/h. Viļņu augstums samazinās, sasniedzot piekrasti.
B: Tiem raksturīgi gari viļņi un maza amplitūda. Ātrums dažreiz sasniedz 700 km/h. Jūras līčos seklūdens zonā strauji palielinās viļņu augstums.
C: Tiem raksturīgi īsi viļņi, kuri veido vairākas viļņu sērijas. Viļņi ļoti strauji sasniedz sauszemi, kuras tuvumā augstums dažreiz pārsniedz 20 metrus.
D: Stipra vēja iekustināti lieli viļņi, kuri pārvietojoties attīstās par ļoti spēcīgiem viļņiem. Jūras līčos seklūdens zonā veidojas gigantiska izmēra viļņi.
Pred: B, Gold: B

Sadedzinot divus litrus gāzveida ogļūdeņraža skābekļa pārākumā, izveidojās astoņi litri ogļskābās gāzes CO2 un desmit litru ūdens tvaika H2O. Visu vielu tilpumi mērīti vienādos apstākļos 100 °C un atmosfēras spiedienā. Kāda ir ogļūdeņraža molekulformula?
A: CH4
B: C2H4
C: C4H10
D: C8H10
Pred: C, Gold: C

Kura fizikālā lieluma precīzai raksturošanai jāno